In [1]:
!pip install qiskit qiskit-optimization

> Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it. 

Given $n$ items, each with an associated weight $w_i$, and bins with maximum weight (capacity) $C$. $x_{ij}$ represent decision variables that equals 1 if item $i$ is put into bin $j$, and $y_j$ = 1 if bin $j$ is used, bounded by the maximum number of bins $n$. The objective is to minimize the number of bins
$$
\quad \sum_{j=1}^{n} y_j
$$

subject to constraints that 
1. 1 item is assigned to 1 bin
$$
\quad \sum_{j=1}^{n} x_{ij} = 1, \quad \forall i = 1, \ldots, n
$$
2. total weight of items respect the bin capacity
$$
\quad \sum_{i=1}^{n} w_i x_{ij} \leq C, \quad \forall j = 1, \ldots, n
$$
3. if a bin is not used $y_j = 0$, no items should be assigned to it, so $x_{ij} = 0$ for all i. 
$$
\quad x_{ij} \leq y_j, \quad \forall i = 1, \ldots, n, \forall j = 1, \ldots, n
$$
We can also modify the right hand side of capacity constraint from C to $C y_j$, which could implies the last constraint, but it is separated to improve the performance of the model.


In [2]:
from docplex.mp.model import Model

def define_ilp(items_weight: list, bin_capacity: float, verbose = False) -> Model:
    """
    Define an integer linear programming model for the bin packing problem.

    Parameters
    ----------
    items_weight : list of float
        Weights of the items.
    bin_capacity : float
        naximum weight that a bin can hold.
    """
    N = len(items_weight)
    model = Model()
    x = {(i, j): model.binary_var() for i in range(N) for j in range(N)}
    y = [model.binary_var() for j in range(N)]
    model.minimize(model.sum(y[j] for j in range(N)))
    for i in range(N):
        model.add_constraint(model.sum(x[i, j] for j in range(N)) == 1)
    for j in range(N):
        model.add_constraint(model.sum(items_weight[i] * x[i, j] for i in range(N)) <= bin_capacity)
    for i in range(N):
        for j in range(N):
            model.add_constraint(x[i, j] <= y[j])
    if verbose:
        print(f"Number of variables: {model.number_of_variables} and constraints: {model.number_of_constraints}")
        print(f"Model objective: {model.get_objective_expr()}")
    return model


> Create a function to transform the ILP model into a QUBO 

The objective function is linear with the form $L x$ and quatradic with the form $x^T Q x$, where x is a vector of binary decision variables and Q is a square matrix of constants. As an alternative to ILP constraints, we introduce quadratic penalties into the objective function. 
1. In general, converting a equality constraint $a x = b$ to a penalty term $(a x - b)^2$ takes the form of
- constant $ C += b^2 $
- linear $ L[x_i] -= 2 coef[x_i] * b $
- quadratic $ Q[x_i, x_j] += coef[x_i] * coef[x_i] $

2. A inequality constraint can be converted to equality form by including slack variables in a binary expansion. The number of slack variables per equation is $log_2(b)$. The equation becomes $(a x + 2^l s_l = b)$, where $s$ is the slack variable with $2^l$ or $b - 2^l$ as the coefficient. The penalty term is then computed as above.

3. The constraint $x_i \leq x_j$ is special can be converted to $x_i - x_j x_i$.

The matrix L and Q is constructed as the first n rows (and columns) are the $x_{ij}$ variables, the next n are the $y_j$ variables, and the rest are the slack variables. The QUBO matrix is the sum of the penalties and objective function. 
For a small system with 2 items with weights 1 and 2 and 2 bins with capacity 3, assuming penalty factor is 1, 
1. the objective function is
$$
Q = x5 + x6
$$
2. Item Assignment Constraint:
rhs = 1, coefs = 1, an example of the penalty term is
$$
x1+x2 == 1
$$
$$
P_1 = (1 - x1 - x2)^2 = 1 - 2 x1 - 2 x2 + x1 x2 + x2 x1
$$

3. Bin Capacity Constraint:
number of slack variables = 2, rhs = 3, coefs = [1, 2, 1, 2], an example of the equality form is
$$
x1 + 2 x3 <= 3,
$$
$$
x1 + 2 x3 + s1 + 2 s2 = 3
$$

4. Bin Assignment Constraint:
$$
x1 <= x5, 
P_5 = x1 - x1 x5
$$

In [3]:
import numpy as np
import math
def constraint_to_penalty(num_total_variables, item_idx_list, penalty_factor, rhs, coeffs: np.ndarray):
    constant = penalty_factor * rhs ** 2
    linear = np.zeros(num_total_variables)
    quadratic = np.zeros((num_total_variables, num_total_variables))
    for item_idx in item_idx_list: # for each bin that item could be placed
        # linear term
        linear[item_idx] -= 2 * penalty_factor * rhs * coeffs[item_idx]
        # quadratic term
        for item_jdx in item_idx_list:
            quadratic[item_idx, item_jdx] += penalty_factor * coeffs[item_idx]* coeffs[item_jdx]
    return constant, linear, quadratic

def define_qubo(items_weight: list, bin_capacity: float, penalty_factor: float = 1.0, verbose = False) -> np.ndarray:
    """
    Define a QUBO model for the bin packing problem.

    Parameters
    ----------
    items_weight : list of float
        Weights of the items.
    bin_capacity : float
        Maximum weight that a bin can hold.
    penalty_factor : float
        Penalty factor for the constraints.
    """
    num_item = len(items_weight)
    item_variables = (0, num_item ** 2)
    bin_variables = (item_variables[1], item_variables[1] + num_item)
    num_slack_variables_per_eq = math.floor(np.log2(bin_capacity))
    last_slack_coeff = None
    if 2 **num_slack_variables_per_eq - 1 < bin_capacity:
        last_slack_coeff = bin_capacity + 1 - 2 ** (num_slack_variables_per_eq)
        num_slack_variables_per_eq += 1
    slack_variables = (bin_variables[1], bin_variables[1] + num_slack_variables_per_eq * num_item)
    num_total_variables = slack_variables[1]
    P2_coeffs = np.zeros(num_total_variables)
    for i in range(num_item):
        for j in range(num_item):
            P2_coeffs[i * num_item + j] = items_weight[i]
            
    for i in range(num_item):
        for l in range(num_slack_variables_per_eq):
            if last_slack_coeff is not None and l == num_slack_variables_per_eq - 1:
                P2_coeffs[slack_variables[0] + i * num_slack_variables_per_eq + l] = last_slack_coeff
            else:
                P2_coeffs[slack_variables[0] + i * num_slack_variables_per_eq + l] = 2 ** l
            
    constant = 0
    linear = np.zeros(num_total_variables)
    quadratic = np.zeros((num_total_variables, num_total_variables))
    # objective: 
    for i in range(bin_variables[0], bin_variables[1]):
        linear[i] = 1
    # constraints:
    # P_1 = (1 - x1 - x2 - ... - xN)^2 = 1 - 2 * (x1 + x2 + ... + xN) + (x1^2 + x2^2 + ... + xN^2)
    for i in range(0, num_item**2, num_item): # for constraints on each item
        item_idx_list = range(i, num_item + i)
        P1_coeffs = np.ones(num_total_variables)
        Pconstant, Plinear, Pquadratic = constraint_to_penalty(num_total_variables, item_idx_list, penalty_factor, 1, P1_coeffs)
        constant += Pconstant
        linear = np.add(linear, Plinear)
        quadratic = np.add(quadratic, Pquadratic)
    # P_2 = (x1 + x2 + ... + xN - y1)^2 = x1^2 + x2^2 + ... + xN^2 - 2 * (x1 * y1 + x2 * y2 + ... + xN * yN) + y1^2
    for b in range(0, num_item): # for each bin 
        item_idx_list = range(b, num_item**2, num_item) 
        slack_idx_list = range(slack_variables[0] + b * num_slack_variables_per_eq, slack_variables[0] + (b + 1) * num_slack_variables_per_eq)
        item_idx_list = list(item_idx_list) + list(slack_idx_list)
        Pconstant, Plinear, Pquadratic = constraint_to_penalty(num_total_variables, item_idx_list, penalty_factor, bin_capacity, P2_coeffs)
        constant += Pconstant
        linear = np.add(linear, Plinear)
        quadratic = np.add(quadratic, Pquadratic)
    # P_3 = x1 - x1x5 + x2 - x2x6 + ... 
    for b in range(0, num_item): # for each bin 
        bin_idx = bin_variables[0] + b
        for item_idx in range(b, num_item**2, num_item): # for each item that could be placed in the bin
            linear[item_idx] += penalty_factor
            # quadratic term
            quadratic[item_idx, bin_idx] -= penalty_factor
    # convert quadratic to upper triangular matrix by adding the lower triangular part
    quadratic += quadratic.T - np.diag(quadratic.diagonal())
    quadratic[np.tril_indices(num_total_variables, -1)] = 0
    if verbose:
        print(f"Total number of variables: {num_total_variables}, item idx {item_variables}, bin idx {bin_variables}, slack idx {slack_variables}")
        print(f"Offset: {constant}, num of linear terms: {np.count_nonzero(linear)}, num of quadratic terms: {np.count_nonzero(quadratic)}")
    return constant, linear, quadratic

> Test your function with specific instances (size small, medium, and big) 

In [4]:
import numpy as np
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

np.random.seed(42)

instances = {
    'small1': {
        'weights': [1, 2], 
        'bin_capacity': 3
    },
    'small2': {
        'weights': [2, 2], 
        'bin_capacity': 4
    },
    'medium': {
        'weights': [1, 2, 1, 2],
        'bin_capacity': 3 # cap the number of slack variables to 2
    },
    'large': {
        'weights': np.random.randint(1, 5, size=8),
        'bin_capacity': 7                     
    }
}
for instance in instances.keys():
    model = define_ilp(instances[instance]['weights'], instances[instance]['bin_capacity'])
    constant, linear, quadratic = define_qubo(instances[instance]['weights'], instances[instance]['bin_capacity'], penalty_factor=3.0, verbose=True)
    test_qp = from_docplex_mp(model)
    conv = QuadraticProgramToQubo(penalty=3.0)
    test_qubo = conv.convert(test_qp)
    test_constant = test_qubo.objective.constant
    test_linear = test_qubo.objective.linear.to_array() 
    test_quadratic = test_qubo.objective.quadratic.to_array()
    if np.isclose(constant, test_constant) and np.allclose(linear, test_linear) and np.allclose(quadratic, test_quadratic):
        print(f"{instance} instance: PASSED")
    else:
        print(f"{instance} instance: FAILED")
        print(f"constant error: {np.abs(constant - test_constant)}")
        print(f"linear error: {np.abs(linear - test_linear)}")
        print(f"quadratic error: {np.abs(quadratic - test_quadratic)}")

Total number of variables: 10, item idx (0, 4), bin idx (4, 6), slack idx (6, 10)
Offset: 60.0, num of linear terms: 10, num of quadratic terms: 26
small1 instance: PASSED
Total number of variables: 12, item idx (0, 4), bin idx (4, 6), slack idx (6, 12)
Offset: 102.0, num of linear terms: 12, num of quadratic terms: 36
small2 instance: PASSED
Total number of variables: 28, item idx (0, 16), bin idx (16, 20), slack idx (20, 28)
Offset: 120.0, num of linear terms: 28, num of quadratic terms: 124
medium instance: PASSED
Total number of variables: 96, item idx (0, 64), bin idx (64, 72), slack idx (72, 96)
Offset: 1200.0, num of linear terms: 96, num of quadratic terms: 816
large instance: PASSED


> Create a Brute Force solver for the QUBO problem and solve the specific instances. 

1. Generate all $2^n$ possible solutions; in practice, we can limit the search space by the constraints.
2. Evaluate the objective function for each combination
3. Track the best solution

For 4 items, total number of variables is 36, and the number of possible solutions is $2^{28} = 2.7 × 10^8$. For 8 items, total number of variables is 96, and the number of possible solutions is $2^{96} = 7.9 × 10^{28}$. The time complexity is $O(2^n)$, which is not feasible for large problems.

In [5]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer

def qubo_brute_force(offset, linear, quadratic):
    from itertools import product
    num_vars = len(linear)
    min_energy = float('inf')
    min_state = None
    for state in product([0, 1], repeat=num_vars):
        energy = offset
        energy += np.dot(linear, state)
        energy += np.dot(state, np.dot(quadratic, state))
        if energy < min_energy:
            min_energy = energy
            min_state = state
    return min_state, min_energy
for instance in ['small1', 'small2']:
    model = define_ilp(instances[instance]['weights'], instances[instance]['bin_capacity'])
    offset, linear, quadratic = define_qubo(instances[instance]['weights'], instances[instance]['bin_capacity'])
    min_state, min_energy = qubo_brute_force(offset, linear, quadratic)
    print(f"{instance}: {min_state} with energy {min_energy}")
    
    # test with numpy minimum eigensolver
    model = define_ilp(instances[instance]['weights'], instances[instance]['bin_capacity'])
    test_qp = from_docplex_mp(model)
    conv = QuadraticProgramToQubo(penalty=3.0)
    test_qubo = conv.convert(test_qp)
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)
    exact_result = exact.solve(test_qubo)
    print(exact_result)

small1: (0, 1, 0, 1, 0, 1, 1, 1, 0, 0) with energy 1.0


AttributeError: 'csr_matrix' object has no attribute 'H'

> To solve the QUBO, use quantum annealing simulators. You can use the Dwave Ocean Framework.
1. convert the QUBO to a BinaryQuadraticModel in dimod
2. solve the BQM with the sampler

In [6]:
!pip install dwave-ocean-sdk
!dwave config create # config file created in github codespace

Using the simplified configuration flow.
Try 'dwave config create --full' for more options.

Updating existing configuration file: /home/codespace/.config/dwave/dwave.conf
Available profiles: defaults
Updating existing profile: defaults
Solver API token [DEV-46f97df115fa978edb8323c48ca219c41d2e7c54]: ^C
Unhandled error: 


In [7]:
def list_to_dict(linear, quadratic):
    linear_dict = {i: linear[i] for i in range(len(linear))}
    quadratic_dict = {(i, j): quadratic[i, j] for i in range(len(quadratic)) for j in range(len(quadratic))}
    return linear_dict, quadratic_dict

In [9]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector
import dimod

def quantum_annealing_qubo(offset, linear, quadratic):
    linear_dict, quadratic_dict = list_to_dict(linear, quadratic)
    bqm = dimod.BinaryQuadraticModel(linear_dict, quadratic_dict, offset, vartype=dimod.BINARY)
    # Submit the QUBO to the quantum annealer
    sampler = EmbeddingComposite(DWaveSampler()) # generak
    sampleset = sampler.sample(bqm, num_reads = 100)
    best_solution = sampleset.first.sample
    best_energy = sampleset.first.energy
    return best_solution, best_energy

for instance in ['small1', 'small2']:
    offset, linear, quadratic = define_qubo(instances[instance]['weights'], instances[instance]['bin_capacity'])
    best_solution, best_energy = quantum_annealing_qubo(offset, linear, quadratic)
    solution_list = [0] * (max(best_solution.keys()) + 1)
    for key, value in best_solution.items():
        solution_list[key] = int(value)
    print(f"best solution: {solution_list} with energy {best_energy}")

best solution: [1, 0, 1, 0, 1, 0, 0, 0, 1, 1] with energy 1.0
best solution: [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1] with energy 1.0


> Use a Quantum Variational approach to solve the QUBO. 
Create multiple Ansantz for tests. 
- The quantum Ansatz is a parametric quantum circuit that can represent possible solutions to the QUBO problem.
Build a function with input being the QUBO and Ansantz. Using a hybrid approach solved the QUBO. 
- The quantum circuit is evaluated by measuring the energy (expectation value of the cost Hamiltonian), and a classical optimizer iteratively adjusts the parameters to minimize the energy.

- define a cost function C 
    - the minimum of C(θ) corresponds to the solution of the problem
    - ‘efficiently estimate’ C(θ) by performing measurements on a quantum computer
    - ‘trainable’, which means that it should be possible to efficiently optimize the parameters θ

- proposes multiple ansatz

proposes multiple ansatz, that is, a quantum operation depending on a set of continuous or discrete parameters θ that can be optimizedtrained in a hybrid quantum-classical loop to solve the optimization task

In [1]:
!pip install pennylane --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 45.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 39.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


> Use QAOA to solve the QUBO. 

Create from scratch a QAOA function. 
as p increases the approximation improves.
1. preparing qubits in an equal superposition over all possible states by applying a Hadamard gate to each qubit
2. in each run of circuit:
    1. in each layer:
        1. apply the problem Hamiltonian to evolve the quantum state, making states with higher costs less favorable
        2. apply the mixer Hamiltonian to explore the solution space. The Pauli-X operator is applied to flip the qubits, helping the algorithm to explore different bitstrings (configurations).
    2. measure the quantum state to get a candidate solution
    3. parameters γ and β are optimized to minimize the cost function.

The cost Hamiltonian $H_c(Z) $ is obtained from the QUBO formulation
$$
H_c(\mathrm{z}) = \sum_{i, j > i}^{n} J_{ij} z_i z_j + \sum_{i=1}^n h_{i}z_i + O
$$
and translated into a parametric unitary gate given by
$$
U(H_c, \gamma_i)=e^{-i \gamma_i H_c},\tag{11} = e^{-i \gamma_i \left( \sum_{i<j}^{n-1} J_{ij}Z_iZ_j + \sum_{i}^{n-1} h_iZ_i\right)}
$$
where $\gamma_i$ is a parameter to be optimized. The term $e^{-i \gamma_i J_{ij}Z_iZ_j}$ is implemented by $RZZ(2 \gamma_i J_{ij})$, and $e^{-i \gamma_i h_iZ_i}$ is implemented by $RZ(2 \gamma_i h_i)$.
The mixer Hamiltonian $H_m(X)$ is a parametric unitary gate given by
$$
U(B, \beta_i)=e^{i \beta_i X},\tag{12}
$$
where $\beta_i$ is a parameter to be optimized. The term $e^{i \beta_i X}$ is implemented by $RX(2 \beta_i)$.

We repeat this sequence of gates p times.

In [1]:
def bin_packing_cost(config, linear, quadratic, offset):
    energy = offset
    for k, v in linear.items():
        energy += v * config[k[0]]
    for k, v in quadratic.items():
        energy += v * config[k[0]] * config[k[1]]
    # Return the total energy of the Ising model
    return energy

from collections import defaultdict
import pennylane as qml

shots = 5000  # Number of samples used
dev = qml.device("default.qubit", shots=shots)


@qml.qnode(dev)
def qaoa_circuit(gammas, betas, h, J, num_qubits):
    wmax = max(
        np.max(np.abs(list(h.values()))), np.max(np.abs(list(h.values())))
    )  # Normalizing the Hamiltonian is a good idea
    p = len(gammas)
    # Apply the initial layer of Hadamard gates to all qubits
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    # repeat p layers the circuit shown in Fig. 1
    for layer in range(p):
        # ---------- COST HAMILTONIAN ----------
        for ki, v in h.items():  # single-qubit terms
            qml.RZ(2 * gammas[layer] * v / wmax, wires=ki[0])
        for kij, vij in J.items():  # two-qubit terms
            qml.CNOT(wires=[kij[0], kij[1]])
            qml.RZ(2 * gammas[layer] * vij / wmax, wires=kij[1])
            qml.CNOT(wires=[kij[0], kij[1]])
        # ---------- MIXER HAMILTONIAN ----------
        for i in range(num_qubits):
            qml.RX(-2 * betas[layer], wires=i)
    return qml.sample()


def samples_dict(samples, n_items):
    """Just sorting the outputs in a dictionary"""
    results = defaultdict(int)
    for sample in samples:
        results["".join(str(i) for i in sample)[:n_items]] += 1
    return results


In [ ]:

# Create the QAOA circuit
def create_qaoa_circuit(n_qubits, p, gamma, beta):
    """
    Create a QAOA circuit with p layers.
    
    Parameters:
    - n_qubits: Number of qubits (items/bins).
    - p: Number of QAOA layers.
    - gamma: List of gamma parameters.
    - beta: List of beta parameters.
    """
    # Create quantum circuit
    qc = QuantumCircuit(n_qubits)
    
    # Step 1: Initialize in superposition (Hadamard on all qubits)
    for qubit in range(n_qubits):
        qc.h(qubit)

    # Step 2: Apply the QAOA layers
    for layer in range(p):
        # Apply the cost Hamiltonian
        for qubit in range(n_qubits):
            qc.rz(2 * gamma[layer], qubit)  # Phase rotation to encode cost

        # Apply the mixer Hamiltonian (Pauli-X rotations)
        for qubit in range(n_qubits):
            qc.rx(2 * beta[layer], qubit)  # X rotation to mix states

    # Step 3: Measure the qubits
    qc.measure_all()

    return qc

# Define the cost function to minimize during classical optimization
def qaoa_cost_function(params, n_qubits, p, backend, shots=1024):
    """
    The function to optimize. Runs the QAOA circuit and calculates the cost.
    
    Parameters:
    - params: A list of parameters (alternating gamma and beta).
    - n_qubits: Number of qubits.
    - p: Number of layers.
    - backend: The quantum backend to execute the circuit on.
    - shots: The number of shots (runs) for each circuit execution.
    """
    gamma = params[:p]
    beta = params[p:]
    
    # Create the QAOA circuit with the given parameters
    qc = create_qaoa_circuit(n_qubits, p, gamma, beta)
    
    # Transpile and execute the circuit
    transpiled_qc = transpile(qc, backend)
    job = execute(transpiled_qc, backend, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    
    # Calculate the cost as the weighted sum of the outcomes
    total_cost = 0
    for bitstring, count in counts.items():
        cost = bin_packing_cost(bitstring)  # Calculate the cost for each outcome
        total_cost += cost * count / shots  # Weighted by the number of times each outcome occurred
    
    return total_cost

# Run QAOA
def run_qaoa(n_qubits, p, backend, shots=1024):
    """
    Run the QAOA optimization.
    
    Parameters:
    - n_qubits: Number of qubits (bins/items in bin packing).
    - p: Number of layers for the QAOA.
    - backend: The quantum backend to execute the circuit on.
    - shots: The number of shots (circuit executions).
    """
    # Initial guesses for gamma and beta parameters
    initial_params = np.random.rand(2 * p)

    # Classical optimization using scipy.optimize.minimize
    result = minimize(qaoa_cost_function, initial_params, args=(n_qubits, p, backend, shots),
                      method='COBYLA', options={'maxiter': 200})
    
    # Get the optimized parameters
    optimized_params = result.x
    gamma_opt = optimized_params[:p]
    beta_opt = optimized_params[p:]
    
    # Return the optimal gamma and beta parameters
    return gamma_opt, beta_opt

In [2]:
n_qubits = 4  # Number of items/bins (for bin packing)
p = 2  # Number of QAOA layers

# Use the Qiskit Aer simulator as the backend
backend = Aer.get_backend('qasm_simulator')

# Run QAOA to optimize parameters
gamma_opt, beta_opt = run_qaoa(n_qubits, p, backend)

# Print optimized parameters
print("Optimized gamma:", gamma_opt)
print("Optimized beta:", beta_opt)

# Create the final QAOA circuit with the optimized parameters
final_qc = create_qaoa_circuit(n_qubits, p, gamma_opt, beta_opt)

# Execute the final circuit to get the solution
transpiled_qc = transpile(final_qc, backend)
job = execute(transpiled_qc, backend, shots=1024)
result = job.result()

# Plot the histogram of results
counts = result.get_counts(final_qc)
print("Final measurement outcomes:", counts)
plot_histogram(counts)

NameError: name 'Aer' is not defined

Compare and analyze the results. 

 time-to-solution (TTS). The TTS means a time
that is needed to a heuristic algorithm to find the solution
(ground state energy) with 99% success probability. It is
given by
TTS = taR99,

What is the difference between QAOA, Quantum Annealing, and Quantum Variational approaches with different Ansatz? 

The QUBO model has emerged as an underpinning of the quantum computing area known as quantum annealing and Fujitsu's digital annealing, and has become a subject of study in neuromorphic computing.

QVE variational optimization of a quantum circuit to minimize the expectation value of a given Hamiltonian. The optimization is performed iteratively, with the quantum circuit parameters updated at each step until the most optimal solution is determined. 

QAOA, on the other hand, is a quantum algorithm that prepares a quantum state that is a superposition of all possible solutions to the problem. The algorithm applies a sequence of unitary operations to the initial state, with the number of operations and their parameters being determined by the problem the QAOA algorithm has been designed to solve.

VQE is better suited for problems that require a high degree of precision, while QAOA is better suited for problems with a large array of initial possibilities. VQE is known to be more efficient than QAOA for problems with a lower variety of initial solutions, as it can converge to the optimal solution faster. However, for problems with a higher variety of initial solutions, VQE may become computationally expensive due to the exponential growth of the required resources. QAOA, on the other hand, is designed to handle problems with a large search space efficiently.

How do the results compare with the brute force approach? 

All tasks were grouped by fuel elements quantity for
demonstrating results (see Fig. 1). We note that the DWave annealer shows good result in problem solving in
the small-size cases (2 possible canisters and 2 elements),
while optimal solution was not found for 6 and more elements problem. The standard deviation of TTS is significantly increase with elements quantity for all methods.
This is caused by an exponential growth of the space of
possible solutions, leading to a decrease in the probability
of finding the optimal solution. As a result, the annealing
process often terminates at a suboptimal point instead
of the ground state. This is especially true for complex
problems that require a large number of variables to be
taken into consideration.


## References
1. Quantum Bridge Analytics I: A Tutorial on Formulating and Using QUBO Models https://arxiv.org/pdf/1811.11538
2. A Quantum Approximate Optimization Algorithm https://arxiv.org/pdf/1411.4028
3. Variational quantum algorithms https://arxiv.org/pdf/2012.09265
4. Quantum and quantum-inspired optimization for solving the minimum bin packing problem https://arxiv.org/pdf/2301.11265
4. https://github.com/qiskit-community/qiskit-optimization/blob/807d48167caf11cd93bec85f26b34614fb7868da/docs/tutorials/02_converters_for_quadratic_programs.ipynb#L16
Comparing VQE and QAOA: Two Quantum Algorithms for Optimization Problems https://www.quantumgrad.com/article/700
https://qiskit-community.github.io/qiskit-optimization/tutorials/01_quadratic_program.html